In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
sparkSession = SparkSession.builder.appName("veriMaratonu").getOrCreate()

In [4]:
rawDF = sparkSession.read.csv("ratings.csv",header=True,inferSchema=True)

In [5]:
rawDF.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [6]:
mlDF= rawDF.drop('timestamp')

In [7]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True)

In [8]:
(trainDF, testDF) = mlDF.randomSplit([0.8,0.2])

In [9]:
trainDF.count()
testDF.count()

20079

In [10]:
model = als.fit(trainDF) #verisi fazla df'yi eklemek daha saglikli

In [11]:
predictDF = model.transform(testDF)

In [12]:
predictDF.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     2|     17|   5.0|  3.956649|
|     2|    144|   3.0| 2.9239998|
|     2|    153|   4.0| 3.1901162|
|     2|    161|   3.0| 3.3164012|
|     2|    165|   3.0|  2.989243|
|     2|    186|   3.0| 2.3650892|
|     2|    265|   5.0| 3.3722694|
|     3|    267|   3.0| 2.6759079|
|     3|    296|   4.5| 3.9780269|
|     2|    296|   4.0|  3.638776|
|     2|    317|   2.0| 2.6716557|
|     2|    405|   2.0|  2.212656|
|     2|    457|   3.0| 3.8219404|
|     2|    474|   2.0| 4.0417333|
|     2|    497|   3.0|   4.34721|
|     2|    527|   4.0| 3.9084399|
|     2|    537|   4.0| 1.9922872|
|     2|    586|   3.0|  2.970127|
|     1|   1029|   3.0|  2.572977|
|     1|   2193|   2.0|  3.086059|
+------+-------+------+----------+
only showing top 20 rows

